In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import argparse
import time
import sys
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
rootDirectory = "/homes/gws/kdeeds/FrequencyBounds"

In [3]:
sys.path.append(rootDirectory + "/BayesCard")
from DataPrepare.schemas import gen_DB0_schema, gen_stats_light_schema
schema = gen_stats_light_schema(rootDirectory +"/Data/Stats/stats_simplified_bayes_card")
ensemble_location = rootDirectory + "/StatObjects/BayesCardEnsembles/"
from DataPrepare.query_prepare_BayesCard import prepare_join_queries
query_filename = rootDirectory + "/Workloads/StatsSubQueriesBayes.sql" 
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path=None, 
                                            query_filename=query_filename, true_card_exist=True)

In [25]:
sys.path.append(rootDirectory + "/Source")
from JoinGraphUtils import *
queryHints = pickle.load(open(rootDirectory +"/StatObjects/TrueCardinality_JOBM.pkl", "rb"))

['MC', 'T']

In [4]:
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path=ensemble_location):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns.append(bn)
            #bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)
with open(query_filename, "rb") as f:
    real_query = f.readlines()
queries = bn_ensemble.parse_query_all(parsed_queries)

10_chow-liu_1.pkl
1_chow-liu_1.pkl
6_chow-liu_1.pkl
8_chow-liu_1.pkl
4_chow-liu_1.pkl
3_chow-liu_1.pkl
5_chow-liu_1.pkl
2_chow-liu_1.pkl
9_chow-liu_1.pkl
0_chow-liu_1.pkl
7_chow-liu_1.pkl


In [61]:
print(bn_ensemble.bns[4].encoding['postLinks.LinkTypeId'])
bn_ensemble.bns[4].attr_type['postLinks.LinkTypeId']

{-100001.16511531256: 0, 1.0: 1, 3.0: 2}


'categorical'

In [65]:
queries[7]

[95474,
 {'bn_index': 4,
  'inverse': False,
  'expectation': [],
  'query': None,
  'n_distinct': None},
 {'bn_index': 0,
  'inverse': False,
  'expectation': ['users.mul_comments.UserId'],
  'query': {'users.users_nn': 0,
   'posts.posts_nn': 0,
   'posts.CreationDate': [0]},
  'n_distinct': {'users.users_nn': array([1]),
   'posts.posts_nn': array([1]),
   'posts.CreationDate': array([4.80341952e-11])}},
 {'bn_index': 0,
  'inverse': True,
  'expectation': [],
  'query': {'users.users_nn': 0,
   'posts.posts_nn': 0,
   'posts.CreationDate': [0]},
  'n_distinct': {'users.users_nn': array([1]),
   'posts.posts_nn': array([1]),
   'posts.CreationDate': array([4.80341952e-11])}},
 {'bn_index': 1,
  'inverse': False,
  'expectation': [],
  'query': {'users.users_nn': 0,
   'comments.comments_nn': 0,
   'comments.Score': 0},
  'n_distinct': {'users.users_nn': array([1]),
   'comments.comments_nn': array([1]),
   'comments.Score': array([1])}},
 {'bn_index': 1,
  'inverse': True,
  'expect

In [49]:
real_query[7]

b"481420||SELECT COUNT(*) FROM comments as c, posts as p, postLinks as pl, users as u WHERE c.UserId = u.Id AND u.Id = p.OwnerUserId AND p.Id = pl.PostId AND c.Score=0 AND p.CreationDate>='2010-09-06 00:58:21'::timestamp AND p.CreationDate<='2014-09-12 10:02:21'::timestamp AND pl.LinkTypeId=1 AND pl.CreationDate>='2011-07-09 22:35:44'::timestamp;\n"

In [ ]:
'query': {'posts.CreationDate': (4237101.0, 131018541.0),
   'postLinks.postLinks_nn': 1,
   'postLinks.CreationDate': (30753344.0, inf),
   'postLinks.LinkTypeId': 1.0},

In [6]:
latency = []
q_errors = []
preds = []
for i, q in enumerate(queries):
    print(f"predicting query no {i}: {real_query[i]} \n")
    tic = time.time()
#    try:
    pred = bn_ensemble.cardinality(q)
    #except:
    #    print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
    #    continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
    preds.append(pred)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

predicting query no 0: b'SELECT COUNT(*) FROM users as u, badges as b WHERE b.UserId= u.Id AND u.UpVotes>=0;||0||79851\n' 

true cardinality 79851, predicted 79851.00000000001 with q-error 1.0000000000000002
predicting query no 1: b"SELECT COUNT(*) FROM badges as b, comments as c, users as u WHERE c.UserId = u.Id AND b.UserId = u.Id AND b.Date<='2014-09-11 14:33:06'::timestamp AND c.Score=0;||1||10220614\n" 

true cardinality 10220614, predicted 5799860.808026548 with q-error 1.7622171183583373
predicting query no 2: b'SELECT COUNT(*) FROM postHistory as ph, comments as c, users as u WHERE ph.UserId = u.Id AND c.UserId = u.Id AND ph.PostHistoryTypeId=1 AND c.Score=0;||2||1458075\n' 

true cardinality 1458075, predicted 14585708.582131343 with q-error 10.003400773027
predicting query no 3: b"SELECT COUNT(*) FROM postHistory as ph, comments as c, users as u WHERE ph.UserId = u.Id AND c.UserId = u.Id AND ph.PostHistoryTypeId=1 AND ph.CreationDate>='2010-09-14 11:59:07'::timestamp;||3||170

In [ ]:
sys.path.append('/home/ziniu.wzn/FSPN/fspn')
from Data_prepare.schemas import gen_stats_light_schema
from Data_prepare.query_prepare_BayesCard import prepare_join_queries
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
hdf_path = "/home/ziniu.wzn/stats/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'

ensemble_location = "/home/ziniu.wzn/stats/FSPN_ensemble/"
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_sub_queries.sql"
#query_filename = "/home/ziniu.wzn/deepdb-public/benchmarks/job-light/sql/job_light_queries.sql"
pairwise_rdc_path = "/home/yuxing.hyx/repository/imdb-benchmark/spn_ensembles/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=True, true_card_exist=True)

In [ ]:
sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path="/home/ziniu.wzn/stats/BayesCard_ensemble/"):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)

In [ ]:
with open(query_filename, "rb") as f:
    real_query = f.readlines()

queries = bn_ensemble.parse_query_all(parsed_queries)


In [ ]:
latency = []
q_errors = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q)
    except:
        print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"predicting query no {i}: {real_query[i]} \n")
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

In [ ]:
parsed_queries[1869]

In [ ]:
q_errors = np.asarray(q_errors)

In [ ]:
np.where(q_errors > 5000)

In [ ]:
q = queries[1869]

In [ ]:
print(len(q))
q

In [ ]:
real_query[1869]

In [ ]:
bn_ensemble.bns[1].table_name

In [ ]:
bn_ensemble.bns[0].

In [ ]:
parsed_queries[1869]

In [ ]:
p, _ = bn_ensemble.bns[1].query(q[1]["query"], n_distinct=q[1]["n_distinct"], return_prob=True)

In [ ]:
p

In [ ]:
p, _ = bn_ensemble.bns[1].expectation(q[1]["query"], e, n_distinct=q[1]["n_distinct"], return_prob=True)

In [ ]:
q[0]*p

In [ ]:
e = ['posts.mul_postHistory.PostId', 'posts.mul_comments.PostId_nn']

In [ ]:
bn_ensemble.bns[1].fanout_attr_inverse

In [ ]:
pred = np.load("/home/ziniu.wzn/stats/est_results/bayesCard_join.npy")

In [ ]:
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_CEB_sub_queries.sql"
with open(query_filename, "r") as f:
    queries = f.readlines()
true = [int(q.split("||")[-1]) for q in queries]
true = np.asarray(true)

In [ ]:
q_errors = np.maximum(true/pred, pred/true)

In [ ]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")

In [ ]:
pred2 = pred + np.random.normal(10, 100, size=2603)
pred2[pred2<=1] = 1

In [ ]:
q_errors = np.maximum(true/pred2, pred2/true)
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")

In [ ]:
np.save("/home/ziniu.wzn/stats/est_results/bayesCard_join_update", pred2)